# Transfer Learning

Datasets computed at high levels of theory are expensive and thus, usually small. 
A model trained on this data might not be able to generalize well to unseen configurations.
Sometimes this can be remedied with transfer learning:
By first training a model on a lot of data from a less expensive level of theory, only small adjustments to the parameters are required to accurately reproduce the potential energy surface of a different level of theory.


Alternatively, the level of theory might not change, but the dataset is extended.
This is the case in learning on the fly scenarios.
For a demonstration of using transfer learning for learning on the fly, see the corresponding example from the [IPSuite documentation](https://ipsuite.readthedocs.io/en/latest/).


apax comes with discriminative transfer learning capabilities out of the box.
In this tutorial we are going to fine tune a model trained on benzene data at the DFT level of theory to CCSDT.



First download the appropriate dataset from the sgdml website.


Transfer learning can be facilitated in apax by adding the path to a pre-trained model in the config.
Furthermore, we can freeze or reduce the learning rate of various components by adjusting the `optimizer` section of the config.

```yaml
optimizer:
    nn_lr: 0.004
    embedding_lr: 0.0
```

Learning rates of 0.0 will mask the respective weights during training steps.
Here, we will freeze the descriptor, reinitialize the scaling and shifting parameters and reduce the learning rate of all other components.

We can now fine tune the model by running
`apax train config.yaml`

In [1]:
from pathlib import Path

import yaml

from apax.utils.datasets import (
    download_benzene_DFT,
    download_md22_benzene_CCSDT,
    mod_md_datasets,
)
from apax.utils.helpers import mod_config

## Acquire Datasets

For this demonstration we will use the DFT and CC versions of the benzene MD17 dataset.
We start by downloading both and saving them in an appropriate format.

In [2]:
# Download DFT Data

data_path = Path("project")
dft_file_path = download_benzene_DFT(data_path)
dft_file_path = mod_md_datasets(dft_file_path)

In [3]:
# Download CCSD(T) Data

data_path = Path("project")
cc_file_path, _ = download_md22_benzene_CCSDT(data_path)
cc_file_path = mod_md_datasets(cc_file_path)

## Pretrain Model

First, we will train a model on the "large" (in relative terms) but less accurate DFT dataset.
A standard model with default optimizers will do just fine.

In [4]:
!apax template train

/home/ms/miniconda3/envs/apax311/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
config_path = Path("config.yaml")

config_updates = {
    "n_epochs": 100,
    "data": {
        "n_train": 1000,
        "n_valid": 200,
        "batch_size": 8,
        "valid_batch_size": 100,
        "experiment": "benzene_dft",
        "directory": "project/models",
        "data_path": str(dft_file_path),
        "energy_unit": "kcal/mol",
        "pos_unit": "Ang",
    },
}
config_dict = mod_config(config_path, config_updates)

with open("config.yaml", "w") as conf:
    yaml.dump(config_dict, conf, default_flow_style=False)

In [6]:
!apax train config.yaml

INFO | 16:25:57 | Running on [cuda(id=0)]
INFO | 16:25:57 | Initializing Callbacks
INFO | 16:25:57 | Initializing Loss Function
INFO | 16:25:57 | Initializing Metrics
INFO | 16:25:57 | Running Input Pipeline
INFO | 16:25:57 | Read data file project/benzene_mod.xyz
INFO | 16:25:57 | Loading data from project/benzene_mod.xyz
INFO | 16:26:06 | Computing per element energy regression.
INFO | 16:26:06 | Initializing Model
INFO | 16:26:06 | initializing 1 models
INFO | 16:26:10 | Initializing Optimizer
INFO | 16:26:10 | Beginning Training
Epochs: 100%|███████████████████████████████████| 100/100 [00:36<00:00,  2.71it/s, val_loss=0.00997]
INFO | 16:26:47 | Finished training


## Baseline CC Training

Next, we require a CC baseline to quantify the effect of pretraining.
As with the DFT dataset, we will only use a small fraction of the data to emphasize the effects in the low-data regime.

In [7]:
config_path = Path("config.yaml")

config_updates = {
    "n_epochs": 100,
    "data": {
        "n_train": 50,
        "n_valid": 10,
        "batch_size": 4,
        "valid_batch_size": 10,
        "experiment": "benzene_cc_baseline",
        "directory": "project/models",
        "data_path": str(cc_file_path),
        "energy_unit": "kcal/mol",
        "pos_unit": "Ang",
    },
}
config_dict = mod_config(config_path, config_updates)

with open("config_cc_baseline.yaml", "w") as conf:
    yaml.dump(config_dict, conf, default_flow_style=False)

In [8]:
!apax train config_cc_baseline.yaml

INFO | 16:26:51 | Running on [cuda(id=0)]
INFO | 16:26:51 | Initializing Callbacks
INFO | 16:26:51 | Initializing Loss Function
INFO | 16:26:51 | Initializing Metrics
INFO | 16:26:51 | Running Input Pipeline
INFO | 16:26:51 | Read data file project/benzene_ccsd_t-train_mod.xyz
INFO | 16:26:51 | Loading data from project/benzene_ccsd_t-train_mod.xyz
INFO | 16:26:51 | Computing per element energy regression.
INFO | 16:26:51 | Initializing Model
INFO | 16:26:51 | initializing 1 models
INFO | 16:26:55 | Initializing Optimizer
INFO | 16:26:55 | Beginning Training
Epochs: 100%|████████████████████████████████████| 100/100 [00:20<00:00,  4.81it/s, val_loss=0.0632]
INFO | 16:27:16 | Finished training


## DFT -> CC Fine Tuning

Finally, we can fine tune a model that was pretrained on DFT data.
The model architecture remains unchanged for all 3 runs.
However, for fine-tuning we need to specify the path to the base model and how to deal with its parameters.
For each parameter group we can choose to freeze, to reset it or to keep training it.
It is certainly advisable to experiment with different strategies, but a good start consists in freezing the embedding layer if the system we transfer to remains the same and resetting the scale-shift layer if the level of theory changes (DFT and CC have different energy scales).

Make sure to carefully inspect the config options below.

In [9]:
config_path = Path("config.yaml")

config_updates = {
    "n_epochs": 100,
    "data": {
        "n_train": 50,
        "n_valid": 10,
        "batch_size": 4,
        "valid_batch_size": 10,
        "experiment": "benzene_cc_ft",
        "directory": "project/models",
        "data_path": str(cc_file_path),
        "energy_unit": "kcal/mol",
        "pos_unit": "Ang",
    },
    "optimizer": {
        "emb_lr": 0.00,  # freeze embedding layer
        "nn_lr": 0.0005,  # lower lr
        "scale_lr": 0.001,  # lower lr
        "shift_lr": 0.005,  # lower lr
    },
    "checkpoints": {
        "base_model_checkpoint": "project/models/benzene_dft",  # pretrained model
        "reset_layers": ["scale_shift"],  # reset scale-shift layer
    },
}
config_dict = mod_config(config_path, config_updates)

with open("config_cc_ft.yaml", "w") as conf:
    yaml.dump(config_dict, conf, default_flow_style=False)

In [10]:
!apax train config_cc_ft.yaml

INFO | 16:27:20 | Running on [cuda(id=0)]
INFO | 16:27:20 | Initializing Callbacks
INFO | 16:27:20 | Initializing Loss Function
INFO | 16:27:20 | Initializing Metrics
INFO | 16:27:20 | Running Input Pipeline
INFO | 16:27:20 | Read data file project/benzene_ccsd_t-train_mod.xyz
INFO | 16:27:20 | Loading data from project/benzene_ccsd_t-train_mod.xyz
INFO | 16:27:20 | Computing per element energy regression.
INFO | 16:27:20 | Initializing Model
INFO | 16:27:20 | initializing 1 models
INFO | 16:27:24 | Initializing Optimizer
INFO | 16:27:24 | loading checkpoint from project/models/benzene_dft/best
INFO | 16:27:24 | Transferring parameters from project/models/benzene_dft
INFO | 16:27:24 | Transferring parameter: radial_fn
INFO | 16:27:24 | Transferring parameter: dense_0
INFO | 16:27:24 | Transferring parameter: dense_0
INFO | 16:27:24 | Transferring parameter: dense_1
INFO | 16:27:24 | Transferring parameter: dense_1
INFO | 16:27:24 | Transferring parameter: dense_2
INFO | 16:27:24 | Tran

As we can see, the fine-tuned model achieves a lower validation loss than the baseline CC model.

How much further can you improve the fine-tuning (or pretraining) setup?